<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Deployment con Dashboard</center></span>

<center>Deployment para Industria</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
1. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Digitalización de documentos](#Digitalización-de-documentos)
* [Matemáticas](#Matemáticas)
* [Transformación de problemas](#Transformación-de-problemas)
* [Tecnología](#Tecnología)
* [Arquitecturas Neuronales Paradigma](#Arquitecturas-Neuronales-Paradigma)

### Cargar Datos

In [7]:
import pandas as pd
datos = pd.read_csv('../Datos/Noticias_Conflicto y Narcotráfico.csv')
display(datos.head(3))
datos.info()

,Titular,Link,Categoría,Fecha,Noticia
0,Las primeras exigencias del Eln ante inicio de...,https://www.eltiempo.com/justicia/conflicto-y-...,Conflicto y Narcotráfico,"18 de noviembre 2022, 07:38 A. M.",El próximo 21 de noviembre se instalará en Car...
1,Así cayeron los dos narcos colombianos más bus...,https://www.eltiempo.com/justicia/conflicto-y-...,Conflicto y Narcotráfico,"16 de noviembre 2022, 06:49 A. M.","En Europa, en especial en España, las autorida..."
2,"Porque 'puede causar cáncer', Gobierno Petro p...",https://www.eltiempo.com/justicia/conflicto-y-...,Conflicto y Narcotráfico,"16 de noviembre 2022, 12:00 A. M.",En un borrador de decreto del Ministerio de Ju...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Titular    339 non-null    object
 1   Link       339 non-null    object
 2   Categoría  339 non-null    object
 3   Fecha      339 non-null    object
 4   Noticia    332 non-null    object
dtypes: object(5)
memory usage: 13.4+ KB


### Hacer Embeddings

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es", device='mps')

In [10]:
docs = datos['Noticia'].values
doc_emb = model.encode(docs,show_progress_bar=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

### Guardar Embeddings

In [15]:
doc_emb_df = pd.DataFrame(doc_emb)
doc_emb_df.to_csv('../Datos/embedding_noticias.csv',index=False)

In [16]:
doc_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.519615,0.385877,0.087688,0.218916,-0.355550,0.051047,0.648467,-0.070190,0.250946,-0.187800,...,-0.033654,-0.092784,0.298336,-0.183551,-0.047961,-0.397322,-0.302257,-0.563499,0.317494,-0.046310
1,0.100707,0.123503,-0.152805,0.221279,-0.111441,-0.108961,0.836357,-0.217561,0.019972,-0.194761,...,-0.450655,0.109862,0.540469,0.049736,-0.475192,-0.726510,0.247290,-0.877658,0.420150,0.210882
2,-0.112571,0.436359,-0.205737,0.257501,-0.019994,-0.082996,0.493205,-0.154947,0.134967,0.086203,...,0.163844,0.052311,1.056586,-0.016971,0.006920,0.007541,-0.644325,-0.582835,0.224875,0.490383
3,-0.125461,0.145932,-0.063221,0.310446,-0.202635,0.021546,0.880170,-0.584733,-0.075835,-0.253557,...,-0.095209,-0.154011,0.467909,-0.305649,-0.092460,-0.566371,0.298161,-0.762232,0.296389,0.387230
4,-0.494786,0.129931,0.320343,0.275328,-0.733692,0.194770,0.527323,0.063607,-0.574659,-0.007934,...,-0.049321,-0.468228,0.565944,-0.185304,-0.262083,-0.536035,-0.151690,-0.197216,0.195686,-0.081830


### Construir Dash

In [ ]:
import re
import dash
import pandas as pd
from dash import dcc
from dash import html
from dash import dash_table
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es", device='mps')
doc_emb = pd.read_csv('../Datos/embedding_noticias.csv',dtype='float32').values

datos = pd.read_csv('../Datos/Noticias_Conflicto y Narcotráfico.csv')

tit = datos["Titular"]
link = datos["Link"]
cat = datos["Categoría"]
fecha = datos["Fecha"]
noti = datos["Noticia"]

def Responder(Consulta):
    
    query_emb = model.encode(Consulta)

    #Compute dot score between query and all document embeddings
    scores = util.cos_sim(query_emb, doc_emb)[0].cpu().tolist()

    #Combine docs & scores
    doc_score_pairs = list(zip(tit,link,cat,fecha,noti,scores))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[5], reverse=True)

    #Output passages & scores
    doc_score_pairs_df = pd.DataFrame(doc_score_pairs, columns=['Titular','Link','Categoría','Fecha','Noticia','Score'])
    return doc_score_pairs_df

##### DASH #####
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
# Define the Dash app
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Mi Dashboard', className="header-title"),
    html.Br(),
    html.P(
    children="Buscador Semántico de Noticias"
    " usando Modelos de Lenguaje Natural."
    " Hecho por Aprendizaje Profundo.",
    className="header-description",
    ),
    html.Br(),
    html.Div(
        
        html.Center([
        dcc.Input(
            id='input_text',
            type='text',
            placeholder = 'Consulta',
            style={'width': '500px','justifyContent':'center'},
            className="input-description"
        ),
        html.Button(
            'Buscar',
            id='submit_button',
            
        ),
                    ])
          ),
    html.Br(),
    html.Div([
        dash_table.DataTable(
            id='table',
            row_selectable='multi',
            selected_rows=[],
            page_size=10,
            style_table={'overflowX': 'auto'},
            style_cell={
            # all three widths are needed
            'textAlign': 'left',
                
            #'minHeight': '100px',
            'height': 'auto',
            #'maxHeight': '280px',
                
            'minWidth': '10px', 
            'width': '180px',
            'maxWidth': '380px',
            'whiteSpace': 'nowrap',
            #"overflow": "hidden",
            "textOverflow": "ellipsis",
            },
            
            fixed_rows= {'headers':True,'data':0},
            
            style_data_conditional=[
            {'height': '20%',
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
                                    ],
          style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
        )
    ]),
    html.Br(),
    html.Div([
        html.Div(id='selected_texts', style={'white-space': 'pre-wrap'},
                 className="output-description")
    ]),
])

@app.callback(
    Output('table', 'data'),
    [Input('submit_button', 'n_clicks')],
    [State('input_text', 'value')]
)
def update_table(n_clicks, input_text):
    if input_text is not None:
        df = Responder(input_text)
        return df.to_dict('records')

@app.callback(
    Output('selected_texts', 'children'),
    [Input('table', 'selected_rows')],
    [State('table', 'data')],
    
)
def display_selected_texts(selected_rows, data):
    
    if data is not None:   
        data = pd.DataFrame.from_dict(data)
        selected_data = data.loc[selected_rows, 'Noticia']
        texts = selected_data.str.replace(r'\s+', ' ',regex=True)
        texts = texts.str.replace(' \n ', "\n\n",regex=True)
        texts = texts.str.replace(' \n', "\n\n",regex=True)
        texts = re.sub("\[|\]|\'",'',str(selected_data.values))
        texts = texts.replace('\\n', "\n")
        texts = texts.replace('\n ', "\n\n")
        #print(texts)
        return html.Div(texts, style={'white-space': 'pre-wrap'})

app.run(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
